In [1]:
!pip install datasets
!pip install evaluate

from tqdm import tqdm, trange
import multiprocessing
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline

import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir("/content/drive/My Drive/LSEG_Dataset")

file_path = "/content/drive/My Drive/LSEG_Dataset/cwe476_Data_op.csv"
df = pd.read_csv(file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df.head()


,code,CWE-476,Label
0,Perl_mro_get_linear_isa(pTHX_ HV *pTHX_Pointe...,True,1
1,handlePRERET(FlowInstruction *FlowInstruction...,True,1
2,GetInetAddr(char *charPointerMarker)\n\n{ str...,True,1
3,do_redirect(action_list_t *action_list_tPoint...,True,1
4,gf_af_new(GF_Compositor *GF_CompositorPointer...,True,1


In [4]:
df = df[['code','Label']]
dataset = {}
dataset['text'] = list(df['code'])
dataset['label'] = list(df['Label'])
ds = Dataset.from_dict(dataset)
ds = ds.train_test_split(test_size = 0.2)

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/8306 [00:00<?, ? examples/s]

Map:   0%|          | 0/2077 [00:00<?, ? examples/s]

In [7]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8306
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2077
    })
})


In [8]:
train_vector = []
test_vector = []
Y_train = []
Y_test = []

In [9]:
for data in tokenized_dataset['train']:
  result_vector = [data['input_ids'][i] if data['attention_mask'][i] else 0 for i in range (len(data['input_ids']))]
  train_vector.append(result_vector)
  Y_train.append(data['label'])


In [10]:
for data in tokenized_dataset['test']:
  result_vector = [data['input_ids'][i] if data['attention_mask'][i] else 0 for i in range (len(data['input_ids']))]
  test_vector.append(result_vector)
  Y_test.append(data['label'])

In [11]:
print(len(train_vector))
print(len(Y_train))

8306
8306


In [12]:
Y_train = np.array(Y_train)

In [13]:
print(Y_train.shape)
print(Y_train)

(8306,)
[0 0 0 ... 0 0 0]


In [14]:
max_length = 512

# Pad the vectors with zeros to make them of the same length
padded_train_vectors = [vector + [0] * (max_length - len(vector)) for vector in train_vector]

print(padded_train_vectors[0])

X_train = np.array(padded_train_vectors)
# X_train = X_train.type(torch.int64)
print(X_train.shape)

[0, 24064, 34118, 1043, 45780, 1215, 15117, 438, 40845, 15228, 338, 33977, 1640, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 42344, 510, 1215, 495, 7976, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1009, 47787, 510, 1215, 495, 7976, 26170, 8007, 10006, 254, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 386, 18997, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 912, 18997, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 16396, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 3189, 50118, 1437, 1437, 1437, 1437

In [15]:
max_length = 512

# Pad the vectors with zeros to make them of the same length
padded_test_vectors = [vector + [0] * (max_length - len(vector)) for vector in test_vector]

print(padded_test_vectors[0])

X_test = np.array(padded_test_vectors)
print(X_test.shape)

[0, 45258, 1215, 46202, 30383, 1640, 27150, 1215, 18801, 15953, 15362, 1009, 27150, 1215, 18801, 15953, 15362, 26170, 8007, 10006, 254, 6, 1437, 50118, 1437, 16224, 1009, 24262, 26170, 8007, 10006, 254, 6, 1717, 2881, 11, 49334, 48739, 6, 50118, 1437, 1717, 1549, 18366, 1215, 2688, 6, 50118, 1437, 16224, 1009, 24262, 26170, 8007, 10006, 254, 6, 1717, 2881, 1009, 257, 2881, 26170, 8007, 10006, 254, 6, 50118, 1437, 1717, 398, 221, 30547, 387, 2629, 6, 1717, 2881, 15408, 4483, 43, 50118, 45152, 50118, 10431, 1594, 1187, 4550, 12571, 2562, 1215, 37056, 13732, 1215, 10612, 1215, 14280, 104, 4322, 50118, 32727, 1215, 28012, 338, 364, 131, 50118, 45258, 7164, 1000, 47006, 50140, 364, 5457, 821, 506, 1215, 34252, 1215, 267, 41191, 1215, 11127, 1640, 24262, 26170, 8007, 10006, 254, 6, 11, 49334, 48739, 6, 359, 49575, 46613, 36097, 6, 359, 49575, 46613, 37009, 6, 359, 49575, 46613, 44546, 1215, 34609, 6, 16224, 26170, 8007, 10006, 254, 6, 1717, 2881, 26170, 8007, 10006, 254, 6, 740, 43820, 46613

In [16]:
print(X_train[0])

[    0 24064 34118  1043 45780  1215 15117   438 40845 15228   338 33977
  1640 50118  1437  1437  1437  1437  1437  1437  1437 10759 42344   510
  1215   495  7976  1437  1437  1437  1437  1437  1437  1437  1437  1437
  1437  1009 47787   510  1215   495  7976 26170  8007 10006   254     6
 50118  1437  1437  1437  1437  1437  1437  1437 10759 30497  1437  1437
  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437
  1437  1437   386 18997     6 50118  1437  1437  1437  1437  1437  1437
  1437 10759 30497  1437  1437  1437  1437  1437  1437  1437  1437  1437
  1437  1437  1437  1437  1437  1437  1437   912 18997     6 50118  1437
  1437  1437  1437  1437  1437  1437 10759 30497  1437  1437  1437  1437
  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437
 16396     6 50118  1437  1437  1437  1437  1437  1437  1437 10759 30497
  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437  1437
  1437  1437  1437  1437  3189 50118  1437  1437  1

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.fit_transform(X_test)

In [18]:
print(X_train_normalized[0])

[ 0.00000000e+00  9.39906740e-01  1.50927741e+00 -7.63195959e-01
  2.05184441e+00 -7.46526106e-01  8.88873485e-02 -7.67715646e-01
  1.66235476e+00  1.04250349e-01 -7.97947264e-01  1.24640070e+00
 -7.13417932e-01  2.25813871e+00 -7.24122213e-01 -7.16917618e-01
 -7.29218211e-01 -7.12824583e-01 -7.19287402e-01 -6.96030391e-01
 -7.09889406e-01 -1.85312772e-01  1.65264868e+00 -7.64774091e-01
 -7.09630436e-01 -7.46707664e-01 -3.26203175e-01 -6.82912585e-01
 -6.91057358e-01 -6.78663827e-01 -6.89458393e-01 -6.64078982e-01
 -6.75787803e-01 -6.74456973e-01 -6.60544472e-01 -6.73640867e-01
 -6.72392294e-01 -6.91869530e-01  1.99166054e+00 -7.19890089e-01
 -6.90266982e-01 -7.22186141e-01 -2.93149073e-01  7.50577026e-01
 -2.77922016e-01 -1.47591586e-01 -7.22074343e-01 -7.47755871e-01
  2.14237762e+00 -6.72273823e-01 -6.66258951e-01 -6.70604453e-01
 -6.63530811e-01 -6.67477420e-01 -6.68053809e-01 -6.73724127e-01
 -1.19163629e-01  1.00740080e+00 -6.63178275e-01 -6.81226665e-01
 -6.70073473e-01 -6.69341

In [19]:
# train_vector =[[ vector/max(sub_vector) for vector in sub_vector ] for sub_vector in train_vector ]
# test_vector =[[ vector/max(sub_vector) for vector in sub_vector ] for sub_vector in test_vector ]

In [20]:
print(train_vector[0])
print(Y_train.shape)
print(Y_train)

[0, 24064, 34118, 1043, 45780, 1215, 15117, 438, 40845, 15228, 338, 33977, 1640, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 42344, 510, 1215, 495, 7976, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1009, 47787, 510, 1215, 495, 7976, 26170, 8007, 10006, 254, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 386, 18997, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 912, 18997, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 16396, 6, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 10759, 30497, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 3189, 50118, 1437, 1437, 1437, 1437

In [21]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0)))

# import SMOTE module from imblearn library
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train_normalized, Y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

Before OverSampling, counts of label '1': 693
Before OverSampling, counts of label '0': 7613 

After OverSampling, the shape of train_X: (15226, 512)
After OverSampling, the shape of train_y: (15226,) 

After OverSampling, counts of label '1': 7613
After OverSampling, counts of label '0': 7613


In [22]:
X_train = torch.tensor(X_train_normalized)
X_test = torch.tensor(X_test_normalized)

In [23]:
# Model Architecture

class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(512, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 16)
        self.fc7 = nn.Linear(16, 8)
        self.fc8 = nn.Linear(8,1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.3)


    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = torch.relu(self.fc5(x))
        x = self.dropout(x)
        x = torch.relu(self.fc6(x))
        x = torch.relu(self.fc7(x))
        x = self.sigmoid(self.fc8(x))
        return x

print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BinaryClassifier().to(device)
print(model)

True
BinaryClassifier(
  (fc1): Linear(in_features=512, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=32, bias=True)
  (fc6): Linear(in_features=32, out_features=16, bias=True)
  (fc7): Linear(in_features=16, out_features=8, bias=True)
  (fc8): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.3, inplace=False)
)


In [24]:

criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.0004)

x_data_tensor = X_train.float().to(device)

Y_train = np.array(Y_train)
y_data_tensor = torch.from_numpy(Y_train).float().to(device)


In [25]:
print(x_data_tensor.shape)
print(y_data_tensor.shape)

print(x_data_tensor[0])

torch.Size([8306, 512])
torch.Size([8306])
tensor([ 0.0000e+00,  9.3991e-01,  1.5093e+00, -7.6320e-01,  2.0518e+00,
        -7.4653e-01,  8.8887e-02, -7.6772e-01,  1.6624e+00,  1.0425e-01,
        -7.9795e-01,  1.2464e+00, -7.1342e-01,  2.2581e+00, -7.2412e-01,
        -7.1692e-01, -7.2922e-01, -7.1282e-01, -7.1929e-01, -6.9603e-01,
        -7.0989e-01, -1.8531e-01,  1.6526e+00, -7.6477e-01, -7.0963e-01,
        -7.4671e-01, -3.2620e-01, -6.8291e-01, -6.9106e-01, -6.7866e-01,
        -6.8946e-01, -6.6408e-01, -6.7579e-01, -6.7446e-01, -6.6054e-01,
        -6.7364e-01, -6.7239e-01, -6.9187e-01,  1.9917e+00, -7.1989e-01,
        -6.9027e-01, -7.2219e-01, -2.9315e-01,  7.5058e-01, -2.7792e-01,
        -1.4759e-01, -7.2207e-01, -7.4776e-01,  2.1424e+00, -6.7227e-01,
        -6.6626e-01, -6.7060e-01, -6.6353e-01, -6.6748e-01, -6.6805e-01,
        -6.7372e-01, -1.1916e-01,  1.0074e+00, -6.6318e-01, -6.8123e-01,
        -6.7007e-01, -6.6934e-01, -6.7434e-01, -6.6786e-01, -6.6240e-01,
        

In [26]:
# Define loss function and optimizer
criterion = nn.BCELoss()

epochs = 10

for epoch in range(epochs):
    running_loss = 0.0

    for inputs, targets in zip(x_data_tensor, y_data_tensor):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(0))
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()

    # Print average loss for the epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(data)}")

print('Finished Training')


Epoch 1, Loss: 687.6959725054259
Epoch 2, Loss: 601.3927271465336
Epoch 3, Loss: 537.750402900748
Epoch 4, Loss: 490.1103193113353
Epoch 5, Loss: 455.53346778964385
Epoch 6, Loss: 408.01839342048146
Epoch 7, Loss: 404.73242056181215
Epoch 8, Loss: 346.95309143774324
Epoch 9, Loss: 323.07523468826116
Epoch 10, Loss: 326.4429385887451
Finished Training


In [33]:
model.eval()
predicted = model(X_test.float().to(device))


In [34]:
print(predicted)

tensor([[1.3590e-02],
        [2.3766e-01],
        [2.3526e-04],
        ...,
        [2.1112e-02],
        [5.6358e-04],
        [8.8310e-03]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [31]:
Y_test = torch.tensor(Y_test).to('cpu')
# predicted = predicted.to('cpu')
# predicted=predicted.detach().numpy()
from sklearn.metrics import classification_report
print(classification_report(Y_test,predicted))

<ipython-input-31-6fdd3672a87b>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_test = torch.tensor(Y_test).to('cpu')


ValueError: Classification metrics can't handle a mix of binary and continuous targets